In [ ]:
import numpy as np

In [ ]:
key_10 = np.load('10_train_key.npy')
x_10 = np.load('10_train_X.npy')
y_10 = np.load('10_train_Y2_hard.npy')

In [ ]:
def summary_group_counts(labels, group_index, num_groups):
    _, num_class = labels.shape
    counts = np.zeros((num_groups, num_class))
    for g in range(num_groups):
        counts[g, :] = np.sum(labels[np.argwhere(group_index == g), :], axis=0)
    return counts

In [ ]:
def split_subjects(sub_ctg_counts, ref_counts, num_group, num_iter):      
    num_sub, _ = sub_ctg_counts.shape
    msub = round(np.ceil(num_sub / num_group))
    d_max = np.inf
    group_index = np.arange(num_sub)
    for _ in range(num_iter):
        grp_id = np.tile(np.arange(num_group), (msub, 1)
                         ).flatten()[np.random.permutation(num_sub)]
        counts = summary_group_counts(sub_ctg_counts, grp_id, num_group)
        d = np.max(np.abs(counts / ref_counts - 1))
        if d < d_max:
            d_max = d
            group_index = grp_id
            grp_ctg_counts = counts
    return group_index, grp_ctg_counts

In [ ]:
n_iter = 10000
n_grp = 8
np.random.seed(479)

sub_10, sub_10_id = np.unique([s.split('_')[0]
                              for s in key_10], return_inverse=True)
ctg_counts_10 = summary_group_counts(y_10, sub_10_id, len(sub_10))

sub_10_grp, sub_10_grp_counts = split_subjects(ctg_counts_10,
                                               np.sum(ctg_counts_10,
                                                      axis=0) / n_grp,
                                               n_grp, n_iter)

for g in range(n_grp):
    isin = np.isin(sub_10_id, np.argwhere(sub_10_grp == g))
    _, c = np.unique(y_10[isin, :].argmax(-1), return_counts=True)
    # print(c)

    np.save(f"HQP{g:d}_X.npy", x_10[isin, :, :]
            [:, None, :, :], allow_pickle=False)
    np.save(f"HQP{g:d}_Y.npy", y_10[isin, :].argmax(-1), allow_pickle=False)
